In [2]:
import pandas as pd
from os import listdir
from pandas.errors import EmptyDataError

In [3]:
wyborcza_articles = []
for filename in listdir('data/wyborcza/articles'):
    try:
        wyborcza_articles.append(pd.read_csv('data/wyborcza/articles/'+filename, header = None))
    except EmptyDataError:
        pass # empty file
wyborcza_articles = pd.concat(wyborcza_articles)
wyborcza_articles.columns = ['url', 'title', 'short', 'long', 'img', 'com']
wyborcza_articles['short'] = wyborcza_articles['short'].str.replace(r'(.|..)\n', '')
wyborcza_articles = wyborcza_articles[~wyborcza_articles['long'].str.contains('W odpowiedzi do @', na = False) == True]
wyborcza_articles = wyborcza_articles[['title', 'short', 'long']]
wyborcza_articles = wyborcza_articles.dropna()
wyborcza_articles = wyborcza_articles[wyborcza_articles['title'].duplicated() == False]
wyborcza_articles = wyborcza_articles[wyborcza_articles['short'] != '0']

In [4]:
gazeta_articles = []
for filename in listdir('data/gazeta/articles'):
    try:
        gazeta_articles.append(pd.read_csv('data/gazeta/articles/'+filename, header = None))
    except EmptyDataError:
        pass # empty file
gazeta_articles = pd.concat(gazeta_articles)
gazeta_articles.columns = ['url', 'title', 'short', 'long', 'img', 'com']
gazeta_articles = gazeta_articles[['title', 'short', 'long']]
gazeta_articles = gazeta_articles[gazeta_articles['title'].duplicated() == False]

# Github issue

In [5]:
# all data
agora = pd.concat([
    gazeta_articles,
    wyborcza_articles,
]).astype(str).values.flatten()

In [8]:
agora[:2]

array(['Zysk Santandera 21,2 proc. w dół',
       'Największy hiszpański banki Santander, który posiada 96 proc. akcji BZ WBK, zarobił w pierwszym półroczu 3,5 mld euro, czyli 21,2 proc. mniej niż rok temu'],
      dtype=object)

In [9]:
test_end = int(0.03 * len(agora))
valid_end = int(0.06 * len(agora))

In [10]:
agora_test = agora[:test_end]
agora_valid = agora[test_end:valid_end]
agora_train = agora[valid_end:]

In [45]:
with open('../my_roberta/my_data/agora.test.raw', 'w') as f:
    f.write('\n'.join(agora_test))
with open('../my_roberta/my_data/agora.valid.raw', 'w') as f:
    f.write('\n'.join(agora_valid))
with open('../my_roberta/my_data/agora.train.raw', 'w') as f:
    f.write('\n'.join(agora_train))

In [ ]:
# encode (copy pased to terminal)
for SPLIT in train valid test; do \
    python -m examples.roberta.multiprocessing_bpe_encoder \
        --encoder-json gpt2_bpe/encoder.json \
        --vocab-bpe gpt2_bpe/vocab.bpe \
        --inputs my_data/agora.${SPLIT}.raw \
        --outputs my_data/agora.${SPLIT}.bpe \
        --keep-empty \
        --workers 60; \
done

In [ ]:
# binarize (copy pased to terminal)
fairseq-preprocess \
    --only-source \
    --srcdict gpt2_bpe/dict.txt \
    --trainpref my_data/agora.train.bpe \
    --validpref my_data/agora.valid.bpe \
    --testpref my_data/agora.test.bpe \
    --destdir data-bin/agora \
    --workers 60

In [ ]:
# train (copy pased to terminal)
fairseq-train --fp16 'data-bin/agora' \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample 512 \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr 0.0005 --warmup-updates 50 \
    --total-num-update 500 \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences 8 --update-freq 32 \
    --max-update 500 --log-format simple --log-interval 1 \
    --restore-file roberta --skip-invalid-size-inputs-valid-test

In [ ]:
# my model (copy pased to terminal)
loaded = hub_utils.from_pretrained(
    model_name_or_path="checkpoints",
    checkpoint_file="checkpoint_best.pt",
    data_name_or_path="data-bin/agora",
    bpe="sentencepiece",
    sentencepiece_vocab=os.path.join("roberta/", "sentencepiece.model"),
    load_checkpoint_heads=True,
    archive_map=RobertaModel.hub_models(),
    cpu=True
)

In [ ]:
roberta = RobertaHubInterface(loaded['args'], loaded['task'], loaded['models'][0])

In [ ]:
# this commands returns predictions but only with numbers (like roberta does not now how to 
# decode them into words)
roberta.fill_mask('Bolesław chrobry urodził się w  <mask>.', topk = 10)